# 使用TensorFlow完成简单的MINST手写识别模型

In [1]:
import tensorflow as tf
import numpy as np
import os
#引入内置的minst数据
from tensorflow.examples.tutorials.mnist import input_data
#自动下载和读取MNIST数据，如果存在则不下载只读取
#这里的mnist是collections.namedtuple('Datasets', ['train', 'validation', 'test'])定义的，也就是其中有train、validation、test3个域
#每个域均是tensorflow自定义的一个类，其中的属性有images(表示图像数据)、labels(对应的数字)、num_examples(样本数量)、
#epochs_completed(迭代完成数量)，有函数next_batch，用于下一个指定size的数据batch
#images、labels为ndarray类型
#关于数据集的更多信息参见tensorflow.contrib.learn.python.learn.datasets.mnist及tensorflow.contrib.learn.python.learn.datasets.base模块
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
#定义预测模型，Node表示不限大小，这里表示的是样本数量不限，因为image被flaten为一行，输入数据为 样本大小×图片像素 的一个2阶tensor
X = tf.placeholder(tf.float32,shape=[None,28*28])
y = tf.placeholder(tf.float32,shape=[None,10])
W = tf.Variable(tf.zeros([28*28,10]))
bias = tf.Variable(tf.zeros([10]))
Y_ = tf.nn.softmax(tf.matmul(X,W)+bias)
#定义预测的准确度
correct_prediction = tf.equal(tf.argmax(y , 1) ,tf.argmax(Y_ , 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
def cal_accuracy( datas, labels):
    return accuracy.eval({ X : datas , y : labels})

In [3]:
#定义损失函数
loss = tf.reduce_mean( -tf.reduce_sum( y  *  tf.log(Y_) , axis=1))
#使用梯度下降
train_ = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

init = tf.global_variables_initializer();


In [4]:
#设置需要记录的值，这里记录每一次迭代的训练集合训练集的代价函数
loss_summary=tf.summary.scalar("loss" , loss)

In [5]:
#开始训练
with tf.Session() as sess:
    sess.run(init);
    #这里我使用两个不同的writer来写入log文件，这是为了在运行tensorboard时，能在同一张图里面看到训练和测试的损失
    train_writer = tf.summary.FileWriter( 'logs/MINIST/run1/train',sess.graph)
    test_writer = tf.summary.FileWriter( 'logs/MINIST/run1/test',sess.graph)
    for i in range(1000):
        batch_x , batch_y = mnist.train.next_batch(100)
        _ , lossS = sess.run([train_ , loss_summary] , {X : batch_x , y : batch_y})
        train_writer.add_summary(lossS,i)
        lossS = sess.run(loss_summary , { X :mnist.test.images , y: mnist.test.labels})
        test_writer.add_summary(lossS,i)
    train_writer.close()
    test_writer.close()
    #打印在测试集上的预测准确率
    print(cal_accuracy(mnist.test.images,mnist.test.labels))

0.9191


## 使用TensorBoard进行模型评估

模型在训练阶段，会使用交叉检验集对模型的超参数（hyper-parameters）进行评估与选择，tensorboard可以很好的展现不同模型之间的差异。

在这里，我们的超参数就是梯度下降的学习率和batch的大小，所以我以这两个参数为例，展示模型的评估和使用tensorboard进行分析。

In [6]:
#设定了3个learn_rate和batch_size
learn_rate = [ 0.05 , 0.2 , 0.5 ]
batch_size = [100 , 200 , 400]
#使用tensorboard时设置logdir为此目录，可以看到所有的运行情况，此时选择需要比较图进行对应的比较则可以看到对比情况
LOG_DIR = "logs\MINIST"

def eval_model(learn_rate,batch_size):
    ##重置参数
    W.assign(tf.zeros([28*28,10]))
    bias.assign(tf.zeros([10]))
    with tf.Session() as sess:
        sess.run(init)
        path = "%.2f-%d" % (learn_rate , batch_size)
        testFilePath = os.path.join(LOG_DIR , path , "test")
        trainFilePath = os.path.join(LOG_DIR,path,"train")
        print("path=%s",testFilePath)
        train_writer = tf.summary.FileWriter(trainFilePath,sess.graph)
        test_writer = tf.summary.FileWriter(testFilePath,sess.graph)
        train1 = tf.train.GradientDescentOptimizer(learn_rate).minimize(loss)
        #为了减少训练时间，设定为200次迭代
        for i in range(200):
            batch_x,batch_y = mnist.train.next_batch(batch_size)
            _ , lossS = sess.run([train1 , loss_summary] , {X : batch_x , y : batch_y})
            train_writer.add_summary(lossS,i)
            lossS = sess.run(loss_summary , { X :mnist.test.images , y: mnist.test.labels})
            test_writer.add_summary(lossS,i)
        train_writer.close()
        test_writer.close()

for r in learn_rate:
    for batch in batch_size:
        print("start %.2f-%d" %(r,batch))
        eval_model(r,batch)

start 0.05-100
path=%s logs\MINIST\0.05-100\test
start 0.05-200
path=%s logs\MINIST\0.05-200\test
start 0.05-400
path=%s logs\MINIST\0.05-400\test
start 0.20-100
path=%s logs\MINIST\0.20-100\test
start 0.20-200
path=%s logs\MINIST\0.20-200\test
start 0.20-400
path=%s logs\MINIST\0.20-400\test
start 0.50-100
path=%s logs\MINIST\0.50-100\test
start 0.50-200
path=%s logs\MINIST\0.50-200\test
start 0.50-400
path=%s logs\MINIST\0.50-400\test
